<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/ons_study_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# @title [CELL 1] Setup + Helper Functions (V86 - Search & Filter Power)
import os
import sys
import re
import csv
import json
import html
import socket
import pytz
import pandas as pd
from ftplib import FTP_TLS
from datetime import datetime

# --- INSTALL TQDM IF MISSING ---
try:
    import tqdm
except ImportError:
    os.system('pip install tqdm')
    import tqdm

print("="*60)
print("      [CELL 1] SETUP LOADED (V86)")
print("      (Includes: Search Bars, Singleton Logic, Data Download)")
print("="*60)

# ==============================================================================
# 1. GLOBAL HELPER FUNCTIONS + HTML ASSETS
# ==============================================================================
TNG_BASE_URL = "https://yates.one-name.net/tng/verticalchart.php?personID="
TNG_SUFFIX = "&tree=tree1&parentset=0&display=vertical&generations=15"

NAV_HTML = """
<style>
nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none}
nav.oldnav li{display:inline-block}
nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px}
nav.oldnav a:hover{background-color:#00838f!important}
@media print { nav.oldnav, #nav-slot, .no-print, .action-btn, .control-panel, .tabs { display: none !important; } }
</style>
<nav class="oldnav"><ul>
<li><a href="/ons-study/research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li>
<li><a href="/ons-study/contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li>
<li><a href="/ons-study/yates_ancestor_register.shtml">DNA Register</a></li>
<li><a href="/ons-study/just-trees.shtml">Trees</a></li>
<li><a href="/ons-study/dna_network.shtml">DNA Network</a></li>
<li><a href="/ons-study/lineage_proof.html">Lineage Proof</a></li>
<li><a href="/ons-study/dna_dossier.html">Forensic Dossier</a></li>
<li><a href="/ons-study/brick_wall_buster.shtml" style="background:#f57f17;color:black !important;">Brick Wall Buster</a></li>
<li><a href="/ons-study/data_glossary.shtml">Data Glossary</a></li>
<li><a href="/ons-study/share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li>
<li><a href="/ons-study/subscribe.shtml" style="background-color:#004d40;">Subscribe</a></li>
</ul></nav>"""

SITE_INFO = """<div class="no-print" style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:90%;border-radius:8px;font-family:sans-serif;"><h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3><p style="color:#333;line-height:1.6;font-size:1.05em;margin-bottom:0;"><strong>Methodology:</strong> This register moves beyond the reliance on single "golden matches" to prove kinship. Instead, it employs <em>Collateral DNA Saturation</em>—a method that blends genealogical reasoning with data-driven logic.</p></div>"""

# V86: Added Table Filtering Logic
JS_CORE = r"""<script type="text/javascript">
(function(){
    // SORTING LOGIC
    function textOf(c){var val = c.getAttribute('data-sort') || c.textContent || c.innerText;return (val || '').replace(/\s+/g,' ').trim().toLowerCase();}
    function sortTable(t,i,d){if(!(t&&t.tBodies&&t.tBodies[0]))return;var tb=t.tBodies[0],r=Array.prototype.slice.call(tb.rows||[]),asc=(d==='asc');r.sort(function(a,b){var A=textOf(a.cells[i]),B=textOf(b.cells[i]),nA=parseFloat(A.replace(/[^0-9.\-]/g,'')),nB=parseFloat(B.replace(/[^0-9.\-]/g,''));if(!isNaN(nA)&&!isNaN(nB))return asc?(nA-nB):(nB-nA);return(A<B)?(asc?-1:1):(A>B)?(asc?1:-1):0;});var f=document.createDocumentFragment();for(var k=0;k<r.length;k++)f.appendChild(r[k]);tb.appendChild(f);}
    function makeSortable(t){if(!(t&&t.tHead&&t.tHead.rows.length))return;var th=t.tHead.rows[0].cells;for(var i=0;i<th.length;i++){(function(idx){var h=th[idx],d='asc';h.style.cursor='pointer';h.onclick=function(){d=(d==='asc')?'desc':'asc';for(var j=0;j<th.length;j++)th[j].innerHTML=th[j].innerHTML.replace(/\s+\(asc\)|\s+\(desc\)/,'');h.innerHTML+=(d==='asc'?' (asc)':' (desc)');sortTable(t,idx,d);};})(i);}}

    // FILTERING LOGIC
    window.filterTable = function() {
        var input = document.getElementById("tableSearch");
        var filter = input.value.toUpperCase();
        var table = document.getElementById("reg-table") || document.querySelector("table.dataframe");
        var tr = table.getElementsByTagName("tr");
        for (var i = 1; i < tr.length; i++) {
            var tdArr = tr[i].getElementsByTagName("td");
            var found = false;
            for (var j = 0; j < tdArr.length; j++) {
                if (tdArr[j]) {
                    var txtValue = tdArr[j].textContent || tdArr[j].innerText;
                    if (txtValue.toUpperCase().indexOf(filter) > -1) {
                        found = true;
                        break;
                    }
                }
            }
            tr[i].style.display = found ? "" : "none";
        }
    }

    function init(){
        var t=document.getElementsByTagName('table');
        for(var i=0;i<t.length;i++) if(/\bsortable\b/.test(t[i].className)) makeSortable(t[i]);
    }
    if(document.readyState==='loading')document.addEventListener('DOMContentLoaded',init,false);else init();
})();
</script>"""

GLOSSARY_CONTENT = """<div style="background:white;padding:25px;border-radius:8px;border:1px solid #ddd;font-family:sans-serif;line-height:1.6;"><h2 style="color:#006064;border-bottom:2px solid #004d40;padding-bottom:10px;">ONS Yates Study: Data Glossary</h2><h3 style="color:#00838f;margin-top:25px;">1. Identity Columns</h3><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Tester-Participant-MASKED (The Trigger):</strong><br>The unique privacy code extracted from the user's NPFX tag.</li><li style="margin-bottom:15px;"><strong>Tester-Participant-Unmasked:</strong><br>The real name of the tester.</li></ul><h3 style="color:#00838f;margin-top:25px;">2. Analysis Terms</h3><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Platinum Standard:</strong><br>Lineages with 30+ matches and 10+ unique sources. Biologically proven.</li><li style="margin-bottom:15px;"><strong>Keystone Tester:</strong><br>A high-value participant (15+ matches) who anchors a specific branch.</li></ul></div>"""

SUBSCRIBE_CONTENT = """<div style="background:white;padding:40px;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);max-width:800px;margin:30px auto;text-align:center;font-family:'Segoe UI',sans-serif;"><h1 style="color:#006064;margin-bottom:10px;">Join the Yates Research Community</h1><p style="color:#555;font-size:1.1em;line-height:1.6;margin-bottom:30px;">Stay connected with the latest breakthroughs in the Yates One-Name Study. Get notified about new DNA groups, lineage verifications, and quarterly reports.</p><div style="background:#e0f2f1;padding:25px;border-radius:8px;border:1px solid #b2dfdb;display:inline-block;"><h3 style="margin-top:0;color:#004d40;">📧 One-Click Subscribe</h3><p style="margin-bottom:20px;">Click below to send a subscription request to our Groups.io list.</p><a href="mailto:yates-one-name-study+subscribe@groups.io?subject=Subscribe" style="display:inline-block;padding:15px 30px;background:#00838f;color:white;text-decoration:none;border-radius:5px;font-weight:bold;font-size:1.1em;box-shadow:0 2px 5px rgba(0,0,0,0.2);">Subscribe Now</a></div><p style="margin-top:30px;font-size:0.9em;color:#777;">Powered by Groups.io. You will receive a confirmation email shortly.</p></div>"""

SHARE_CONTENT = """<div style="max-width:900px;margin:30px auto;font-family:'Segoe UI',sans-serif;line-height:1.6;color:#333;"><div style="text-align:center;margin-bottom:40px;"><h1 style="color:#0277bd;margin-bottom:10px;">Share Your Ancestry DNA Matches</h1><p style="font-size:1.1em;color:#555;">Ancestry provides a built-in sharing feature that allows you to grant limited access to your DNA matches <strong>without sharing your personal account details</strong>. You remain in full control of your account at all times.</p></div><div style="display:grid;grid-template-columns:1fr 1fr;gap:30px;margin-bottom:30px;"><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #0277bd;"><h3 style="color:#0277bd;margin-top:0;">How Sharing Works</h3><p>From your AncestryDNA <strong>Settings</strong> page, you may invite another individual by email and assign one of the following roles:</p><ul style="padding-left:20px;"><li><strong>Viewer</strong> (Read only)</li><li><strong>Collaborator</strong> (Recommended for Study)</li><li><strong>Manager</strong> (Full Control)</li></ul></div><div style="background:#e3f2fd;padding:25px;border-radius:8px;border:1px solid #90caf9;"><h3 style="color:#01579b;margin-top:0;">Privacy & Control</h3><p>This sharing arrangement provides <strong>limited access only</strong>. It does not allow changes to your account and does not expose your personal details.</p><p><strong>You may revoke access at any time through Ancestry.</strong></p></div></div><div style="background:white;padding:30px;border-radius:8px;border:1px solid #ddd;box-shadow:0 4px 15px rgba(0,0,0,0.05);"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">How to Share for the Yates One-Name Study</h2><ol style="font-size:1.1em;line-height:1.8;padding-left:25px;"><li>Open your <strong>AncestryDNA Settings</strong>.</li><li>Scroll to the section labeled "DNA Result Sharing" and click <strong>Invite</strong>.</li><li>Send the invitation to <strong>Ron Yates</strong> at: <br><span style="background:#fff3e0;padding:5px 10px;border-radius:4px;font-weight:bold;color:#e65100;font-family:monospace;font-size:1.2em;">yatesvilleron@gmail.com</span></li><li>Assign the role: <strong>Collaborator</strong>.</li></ol><div style="background:#fffde7;border-left:5px solid #fbc02d;padding:15px;margin-top:20px;font-size:0.95em;"><strong>Why Collaborator?</strong> The Collaborator role allows Ron to review shared matches and create small internal groups (colored dots) to identify which matches have been reviewed and which have contributed evidence to the Yates One-Name Study.</div></div><div style="margin-top:40px;"><h3 style="color:#006064;">What Happens Next?</h3><p>After sharing, you will receive an invitation to subscribe to the <strong>Yates One-Name Study Groups.io mailing list</strong>, where DNA proof summaries and study findings are shared.</p><h3 style="color:#006064;">Reciprocal Sharing (Optional)</h3><p>If you are interested in viewing Ron’s DNA matches, simply let him know. When a direct match exists, that relationship will be reflected in the study findings.</p></div></div>"""

THEORY_CONTENT = """<div style="max-width:1000px;margin:30px auto;font-family:'Segoe UI',sans-serif;line-height:1.6;color:#333;"><h1 style="text-align:center;color:#004d40;font-size:2.5em;margin-bottom:10px;">The Yates DNA Strategy</h1><p style="text-align:center;font-size:1.2em;color:#666;margin-bottom:40px;">Moving beyond traditional Y-DNA to solve modern genealogical mysteries.</p><div style="display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:30px;margin-bottom:40px;"><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #00838f;"><h2 style="color:#006064;margin-top:0;">The Autosomal Revolution</h2><p>Traditional one-name studies rely almost exclusively on Y-DNA to trace the paternal surname line. While valuable for deep history, this approach ignores 50% of our ancestors at every generation.</p><p><strong>Our Focus:</strong> We utilize <strong>Autosomal DNA (atDNA)</strong>—inherited from both parents—to verify connections across <em>all</em> branches. This allows us to:</p><ul style="padding-left:20px;color:#444;"><li>Bridge the "Gender Gap" by tracing female descendants.</li><li>Verify paper trails for the last 300 years (Genealogical Time).</li><li>Cluster "Floating" Yates families into their correct lines.</li></ul></div><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #f9a825;"><h2 style="color:#f57f17;margin-top:0;">Collateral Saturation</h2><p>A single DNA match can be luck. Ten matches is a statistic. <strong>Thirty matches is a fact.</strong></p><p>We employ a technique called <strong>Collateral Saturation</strong>. We don't look for one "Golden Match." We analyze groups of matches from independent cousin lines. When descendants from four different children of <em>William & Mary Yates</em> all share DNA with you, the relationship is biologically confirmed.</p><div style="text-align:center;margin-top:20px;"><a href="dna_network.shtml" style="display:inline-block;padding:10px 20px;background:#f9a825;color:#333;text-decoration:none;border-radius:4px;font-weight:bold;">View the Network</a></div></div></div><div style="background:#e0f2f1;padding:30px;border-radius:8px;border:1px solid #b2dfdb;margin-bottom:40px;"><h2 style="color:#004d40;margin-top:0;text-align:center;">From Theory to Tools</h2><p style="text-align:center;max-width:700px;margin:0 auto 20px auto;">We have built a suite of forensic tools to visualize this data. Instead of raw spreadsheets, we offer interactive dashboards to prove your connection.</p><div style="display:flex;flex-wrap:wrap;justify-content:center;gap:15px;margin-top:20px;"><a href="ons_yates_dna_register.shtml" style="background:#006064;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">The Register</a><a href="lineage_proof.html" style="background:#00838f;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">Proof Engine</a><a href="dna_dossier.html" style="background:#00acc1;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">Forensic Dossier</a></div></div><div style="background:#f5f5f5;padding:20px;border-radius:8px;border:1px solid #ddd;"><h3 style="color:#555;margin-top:0;">Legacy Data: Y-DNA Haplogroups</h3><p style="font-size:0.9em;color:#666;margin-bottom:15px;">Y-DNA is the backbone of deep ancestry (27,000 BCE to 1600 AD). While not our primary focus for recent genealogy, we maintain a detailed record of the Yates Y-Chromosome mutations (R-M207 through FT266579).</p><a href="https://yates.one-name.net/gengen/dna_proof_y.htm" style="color:#006064;font-weight:bold;text-decoration:none;">&raquo; View Detailed Y-DNA Findings</a></div></div>"""

def make_page(title, content, count, view_type="", extra="", stats_bar=""):
    nav_blk = ""
    if view_type in ['ancestor', 'participant', 'tree_az', 'tree_za', 'proof', 'network', 'dossier', 'subscribe', 'share', 'buster', 'singleton']:
        nav_blk = SITE_INFO
    if view_type == 'subscribe' or view_type == 'theory' or view_type == 'share':
        nav_blk = ""

    toggle = ""
    print_btn = ""
    search_bar = ""

    # V86: Add Search Bar for Registers and Singletons
    if view_type in ['ancestor', 'participant', 'singleton']:
        search_bar = """<div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;"><input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;"></div>"""

    if view_type in ['ancestor', 'participant', 'singleton']:
        view_name = "Register"
        if view_type == 'singleton': view_name = "Singleton List"
        print_btn = f"""<div class="no-print" style="text-align:center;margin-bottom:15px;"><button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print {view_name}</button></div>"""

    if view_type == 'ancestor':
        toggle = f"""<div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="font-weight:bold;color:#006064;">By Ancestral Line</a> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="color:#00acc1;text-decoration:none;">By Participant Name</a></div>"""
    elif view_type == 'participant':
        toggle = f"""<div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="color:#00acc1;text-decoration:none;">By Ancestral Line</a> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="font-weight:bold;color:#006064;">By Participant Name</a></div>"""
    elif 'tree' in view_type:
        za = f'<span style="font-weight:bold;color:#000;">Z-A</span>' if 'za' in view_type else f'<a href="just-trees.shtml" style="color:#006064;text-decoration:underline;">Z-A</a>'
        az = f'<span style="font-weight:bold;color:#000;">A-Z</span>' if 'az' in view_type else f'<a href="just-trees-az.shtml" style="color:#006064;text-decoration:underline;">A-Z</a>'
        toggle = f"""<div class="no-print" style="text-align:center;font-family:sans-serif;font-size:16px;margin:15px 0 10px 0;">Individual Yates Family trees: &nbsp; {za} &nbsp;|&nbsp; {az}</div>"""

    return f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>{title}</title><link rel="stylesheet" href="partials_unified.css"><link rel="stylesheet" href="dna_tree_styles.css">{extra}</head><body id="top"><div class="wrap"><h1 class="centerline">{title}</h1><div id="nav-slot">{stats_bar}{NAV_HTML}</div>{nav_blk}{search_bar}{print_btn}{toggle}{content}{JS_CORE}</div></body></html>"""

print("✅ Cell 1 Loaded! (Search/Filter Logic Added)")

      [CELL 1] SETUP LOADED (V86)
      (Includes: Search Bars, Singleton Logic, Data Download)
✅ Cell 1 Loaded! (Search/Filter Logic Added)


In [29]:
# @title [CELL 2] The Asset Manager (V114 - Local Priority)
def fetch_assets():
    print("="*60)
    print("      [CELL 2] ASSET MANAGER STARTING...")
    print("      Logic: Local Upload > Server Download")
    print("="*60)

    import os
    from ftplib import FTP_TLS
    from google.colab import userdata

    # --- CONFIGURATION ---
    SEARCH_PATHS = [
        "/tng/gedcom",           # Priority 1: User specified
        "/public_html/tng/gedcom",
        "/ons-study",
        "/public_html/ons-study",
        "/"
    ]
    KEY_FILENAME = "match_to_unmasked.csv"

    # -------------------------------------------------------
    # STEP 1: CHECK FOR MANUAL UPLOAD (PRIORITY)
    # -------------------------------------------------------
    print("[STEP 1] Checking local storage...")
    local_files = os.listdir('.')
    local_geds = [f for f in local_files if f.lower().endswith(".ged") and "unmasked" not in f.lower()]

    ged_ready = False

    if local_geds:
        # Sort by newest, just in case multiple exist
        local_geds.sort(key=lambda x: os.path.getmtime(x), reverse=True)
        print(f"    ✅ FOUND LOCAL GEDCOM: {local_geds[0]}")
        print("    --> Skipping server download. Using manual upload.")
        ged_ready = True
    else:
        print("    - No local GEDCOM found. Proceeding to server...")

    # Check for Key File
    key_ready = False
    if os.path.exists(KEY_FILENAME):
        print(f"    ✅ FOUND LOCAL KEY: {KEY_FILENAME}")
        key_ready = True

    # If we have both, we can exit early!
    if ged_ready and key_ready:
        print("\n🟢 SYSTEM READY: All assets found locally.")
        return

    # -------------------------------------------------------
    # STEP 2: DOWNLOAD FROM SERVER (FALLBACK)
    # -------------------------------------------------------
    print("\n[STEP 2] Connecting to Server (Fallback)...")
    try:
        HOST = userdata.get("FTP_HOST")
        USER = userdata.get("FTP_USER")
        PASS = userdata.get("FTP_PASS")

        ftps = FTP_TLS()
        ftps.connect(HOST, 21)
        ftps.auth()
        ftps.login(USER, PASS)
        ftps.prot_p()
        print("    ✅ Secured Connection Established.")

        # A. Download GEDCOM (If we didn't find one locally)
        if not ged_ready:
            print("    🔎 Hunting for GEDCOM on server...")
            for folder in SEARCH_PATHS:
                try:
                    ftps.cwd(folder)
                    files = ftps.nlst()
                    server_geds = [f for f in files if f.lower().endswith(".ged") and "unmasked" not in f.lower()]

                    if server_geds:
                        target = server_geds[0]
                        print(f"       Found: {target} in {folder}")
                        with open(target, "wb") as f:
                            ftps.retrbinary(f"RETR {target}", f.write)
                        print(f"       ⬇️ DOWNLOAD COMPLETE: {target}")
                        ged_ready = True
                        break
                except: continue

        # B. Download Key File (If we didn't find one locally)
        if not key_ready:
            print(f"    🔎 Hunting for {KEY_FILENAME}...")
            # Reset CWD search or assume root/study folders
            for folder in SEARCH_PATHS:
                try:
                    ftps.cwd(folder)
                    if KEY_FILENAME in ftps.nlst():
                        with open(KEY_FILENAME, "wb") as f:
                            ftps.retrbinary(f"RETR {KEY_FILENAME}", f.write)
                        print(f"       ⬇️ DOWNLOAD COMPLETE: {KEY_FILENAME}")
                        key_ready = True
                        break
                except: continue

        ftps.quit()

    except Exception as e:
        print(f"    ❌ Connection/Download Error: {e}")

    # -------------------------------------------------------
    # FINAL STATUS
    # -------------------------------------------------------
    print("-" * 60)
    if ged_ready:
        print("🟢 READY: GEDCOM is available for processing.")
    else:
        print("🔴 CRITICAL: No GEDCOM found (Locally or on Server).")
        print("   Action: Please upload a .ged file to the file browser on the left.")

fetch_assets()

      [CELL 2] ASSET MANAGER STARTING...
      Logic: Local Upload > Server Download
[STEP 1] Checking local storage...
    - No local GEDCOM found. Proceeding to server...
    ✅ FOUND LOCAL KEY: match_to_unmasked.csv

[STEP 2] Connecting to Server (Fallback)...
    ✅ Secured Connection Established.
    🔎 Hunting for GEDCOM on server...
       Found: yates_study_2025.ged in /tng/gedcom
       ⬇️ DOWNLOAD COMPLETE: yates_study_2025.ged
------------------------------------------------------------
🟢 READY: GEDCOM is available for processing.


In [30]:
# @title [CELL 3] The Data Engine (V102 - Dates in Lineage)
def run_engine():
    print("="*60)
    print("      [CELL 3] ENGINE STARTING (V102)")
    print("      (Features: Dates embedded in Lineage Paths)")
    print("="*60)

    import os
    import sys
    import re
    import csv
    from ftplib import FTP_TLS

    # --- CONFIGURATION ---
    HOST = os.environ.get("FTP_HOST", "").strip()
    USER = os.environ.get("FTP_USER", "").strip()
    PASS = os.environ.get("FTP_PASS", "").strip()
    REMOTE_SUBDIR = "ons-study"

    KEY_FILE       = "match_to_unmasked.csv"
    UNMASKED_FILE  = "yates_study_2025_UNMASKED.ged"
    CSV_DB         = "engine_database.csv"

    # ---------------------------------------------------------
    # STEP 0: CHECK & DOWNLOAD ASSETS
    # ---------------------------------------------------------
    print("\n[STEP 0] Checking for Required Files...")
    local_geds = [f for f in os.listdir('.') if f.lower().endswith('.ged') and 'unmasked' not in f.lower()]
    local_ged_exists = len(local_geds) > 0
    local_key_exists = os.path.exists(KEY_FILE)

    if not local_ged_exists or not local_key_exists:
        print("    ⚠️ Missing files locally. Initiating Server Download...")
        try:
            if not HOST:
                from google.colab import userdata
                HOST = userdata.get("FTP_HOST")
                USER = userdata.get("FTP_USER")
                PASS = userdata.get("FTP_PASS")

            ftps = FTP_TLS()
            ftps.connect(HOST, 21)
            ftps.auth()
            ftps.login(USER, PASS)
            ftps.prot_p()

            def download_from_server(filename, target_name=None):
                if not target_name: target_name = filename
                paths = [f"/{REMOTE_SUBDIR}/{filename}", f"/public_html/{REMOTE_SUBDIR}/{filename}", f"/{filename}"]
                for p in paths:
                    try:
                        with open(target_name, "wb") as f:
                            ftps.retrbinary(f"RETR {p}", f.write)
                        print(f"    ✅ Downloaded: {filename}")
                        return True
                    except: continue
                return False

            if not local_key_exists: download_from_server(KEY_FILE)
            if not local_ged_exists:
                print("    🔍 Searching server for GEDCOM files...")
                ged_found = False
                search_dirs = [f"/{REMOTE_SUBDIR}", "/public_html", "/"]
                for d in search_dirs:
                    try:
                        ftps.cwd(d)
                        files = ftps.nlst()
                        geds = [g for g in files if g.lower().endswith('.ged') and 'unmasked' not in g.lower()]
                        if geds:
                            best_ged = geds[0]
                            print(f"    found {best_ged} in {d}")
                            with open(best_ged, "wb") as f:
                                ftps.retrbinary(f"RETR {best_ged}", f.write)
                            print(f"    ✅ Auto-Downloaded GEDCOM: {best_ged}")
                            ged_found = True
                            break
                    except: continue
                if not ged_found: print("    ❌ CRITICAL: No GEDCOM file found on server or locally.")
            ftps.quit()
        except Exception as e: print(f"    ⚠️ Download Error: {e}")
    else: print("    ✅ Local files found. Priority 1.")

    # ---------------------------------------------------------
    # STEP 1: LOAD GEDCOM
    # ---------------------------------------------------------
    ged_files = [f for f in os.listdir('.') if f.lower().endswith('.ged') and 'unmasked' not in f.lower()]
    if not ged_files:
        print("\n[CRITICAL ERROR] No GEDCOM file found anywhere!")
        return False
    ged_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    DEFAULT_GEDCOM = ged_files[0]
    print(f"[INFO] Using GEDCOM: {DEFAULT_GEDCOM}")

    # ---------------------------------------------------------
    # STEP 2: LOAD KEYS
    # ---------------------------------------------------------
    print("\n[STEP 2] Unmasking Participants...")
    unmask_map = {}
    if os.path.exists(KEY_FILE):
        try:
            with open(KEY_FILE, mode='r', encoding='utf-8-sig', errors='replace') as f:
                reader = csv.reader(f)
                for row in reader:
                    if len(row) < 2: continue
                    code = row[0].strip().lower()
                    name = row[1].strip()
                    if code and name and code != "code": unmask_map[code] = name
            print(f"    - Loaded {len(unmask_map)} privacy keys.")
        except Exception as e: print(f"    - Error reading key file: {e}")
    else: print("    [CRITICAL WARNING] No Key File found! Unmasking will fail.")

    def resolve_mask_code_greedy(payload):
        m = re.search(r'(\d+)\s*&?\s*([^ \t\n\r\f\v]+)', payload)
        if m: return m.group(2).lower()
        return None
    def resolve_name(code): return unmask_map.get(code, code)

    # ---------------------------------------------------------
    # STEP 3: GENERATE UNMASKED GEDCOM STREAM
    # ---------------------------------------------------------
    with open(DEFAULT_GEDCOM, 'r', encoding='utf-8', errors='replace') as fin, \
         open(UNMASKED_FILE, 'w', encoding='utf-8') as fout:
        buffer_lines = []; real_name = None
        for line in fin:
            if line.startswith("0 @"):
                if buffer_lines:
                    for bl in buffer_lines:
                        if bl.startswith("1 NAME") and real_name: fout.write(f"1 NAME {real_name}\n")
                        else: fout.write(bl)
                buffer_lines = [line]; real_name = None
            else:
                buffer_lines.append(line)
                if line.startswith("1 NPFX"):
                    parts = line.split(" ", 2)
                    if len(parts) > 2:
                        code = resolve_mask_code_greedy(parts[2].strip())
                        if code: real_name = resolve_name(code)
        if buffer_lines:
            for bl in buffer_lines:
                if bl.startswith("1 NAME") and real_name: fout.write(f"1 NAME {real_name}\n")
                else: fout.write(bl)
    print(f"    - Generated {UNMASKED_FILE}")

    # ---------------------------------------------------------
    # STEP 4: TRACE LINEAGES
    # ---------------------------------------------------------
    print("\n[STEP 3] Tracing Lineages...")
    individuals = {}; families = {}
    def clean_name(raw): return raw.replace("/", "").strip()
    def is_yates(name_str):
        if not name_str: return False
        n = name_str.lower()
        return "yates" in n or "yeates" in n or "yate" in n
    def extract_year(date_str):
        if not date_str: return ""
        m = re.search(r'\d{4}', date_str)
        return m.group(0) if m else ""

    current_id = None; current_fam = None; current_tag = None
    with open(UNMASKED_FILE, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip(); parts = line.split(" ", 2)
            if len(parts) < 2: continue
            level, tag = parts[0], parts[1]
            payload = parts[2] if len(parts) > 2 else ""

            if level == "0" and "INDI" in payload:
                current_id = tag.replace("@", "")
                individuals[current_id] = {"name": "Unknown", "parents_fam": None, "spouse_fams": [], "mask_code": "", "cm": 0, "birt": "", "deat": ""}
                current_fam = None; current_tag = "INDI"
            elif current_id and level != "0":
                if tag == "NAME": individuals[current_id]["name"] = clean_name(payload)
                elif tag == "FAMC": individuals[current_id]["parents_fam"] = payload.replace("@", "")
                elif tag == "FAMS": individuals[current_id]["spouse_fams"].append(payload.replace("@", ""))
                elif tag == "NPFX":
                    code = resolve_mask_code_greedy(payload)
                    if code: individuals[current_id]["mask_code"] = code
                    m = re.search(r'^(\d+)|(\d+)\s*cM', payload, re.IGNORECASE)
                    if m: individuals[current_id]["cm"] = int(m.group(1) if m.group(1) else m.group(2))
                elif tag == "BIRT": current_tag = "BIRT"
                elif tag == "DEAT": current_tag = "DEAT"
                elif tag == "DATE" and current_tag in ["BIRT", "DEAT"]:
                    year = extract_year(payload)
                    if current_tag == "BIRT": individuals[current_id]["birt"] = year
                    if current_tag == "DEAT": individuals[current_id]["deat"] = year
                    current_tag = None

            if level == "0" and "FAM" in payload:
                current_fam = tag.replace("@", "")
                families[current_fam] = {"husb": None, "wife": None}
                current_id = None
            elif current_fam and level != "0":
                if tag == "HUSB": families[current_fam]["husb"] = payload.replace("@", "")
                elif tag == "WIFE": families[current_fam]["wife"] = payload.replace("@", "")

    print(f"    - Parsed {len(individuals)} individuals.")

    # V102: FORMAT DATE SPAN
    def format_date_span(uid):
        if not uid or uid not in individuals: return ""
        b = individuals[uid]["birt"]; d = individuals[uid]["deat"]
        if not b and not d: return ""
        return f"({b if b else '?'} - {d if d else '?'})"

    def climb_yates_line_ids(start_id):
        curr = start_id; path_names = []; path_ids = []
        while curr:
            person = individuals.get(curr)
            if not person: break
            # V102: Append dates to name here
            dates = format_date_span(curr)
            full_display = f"{person['name']} {dates}".strip()
            path_names.append(full_display)
            path_ids.append(curr)

            fam_id = person["parents_fam"]
            if not fam_id or fam_id not in families: break
            fam = families[fam_id]
            dad_id, mom_id = fam["husb"], fam["wife"]
            dad_name = individuals.get(dad_id, {}).get("name", ""); mom_name = individuals.get(mom_id, {}).get("name", "")
            if is_yates(dad_name) and not is_yates(mom_name): curr = dad_id
            elif is_yates(mom_name) and not is_yates(dad_name): curr = mom_id
            else: curr = dad_id if dad_id else mom_id
        return curr, path_names, path_ids

    def analyze_lineage_deep(start_id):
        # V102: Track rich names in queue
        # Queue: (current_id, path_names, path_ids)
        queue = [(start_id, [], [])]; visited = set()
        while queue:
            curr, path_from_start, ids_from_start = queue.pop(0)
            if curr in visited: continue
            visited.add(curr)
            person = individuals.get(curr)
            if not person: continue

            if is_yates(person["name"]):
                top_id, climb_names, climb_ids = climb_yates_line_ids(curr)
                # Reconstruct full line
                # climb_names are [Curr+Dates, Parent+Dates...]
                # path_from_start are [Child+Dates, Grandchild+Dates...]
                full_line_names = list(reversed(climb_names)) + list(reversed(path_from_start))
                full_line_ids = list(reversed(climb_ids)) + list(reversed(ids_from_start))

                top_name_pure = individuals.get(top_id, {}).get("name", "Unknown");
                top_dates = format_date_span(top_id)
                spouse_name = "missing"; spouse_dates = ""
                top_person = individuals.get(top_id)
                if top_person:
                    for fid in top_person["spouse_fams"]:
                        if fid in families:
                            f = families[fid]; spouse_id = None
                            if f["husb"] == top_id: spouse_id = f["wife"]
                            elif f["wife"] == top_id: spouse_id = f["husb"]
                            if spouse_id and spouse_id in individuals:
                                spouse_name = individuals[spouse_id]["name"]; spouse_dates = format_date_span(spouse_id); break
                if "unknown" in spouse_name.lower(): spouse_name = "missing"
                pair_dated = f"{top_name_pure} {top_dates} & {spouse_name} {spouse_dates}" if spouse_name != "missing" else f"{top_name_pure} {top_dates}"
                pair_simple = f"{top_name_pure} & {spouse_name}" if spouse_name != "missing" else f"{top_name_pure}"

                # V102: Ensure top pair is nicely formatted in the list
                rich_lineage_list = list(full_line_names)
                # rich_lineage_list[0] is currently "Name (Dates)". We might want to replace it with the Pair if user prefers,
                # but "Lineage" usually implies the direct blood line. Keeping it as single person + dates is cleaner for the "Arrow" string.

                lineage = " -> ".join(rich_lineage_list)
                id_path_str = ",".join(full_line_ids)
                clean_top = re.sub(r'[^a-zA-Z0-9]', '', top_name_pure)
                clean_spouse = re.sub(r'[^a-zA-Z0-9]', '', spouse_name) if spouse_name != "missing" else "ZZZ"
                sort_key = f"{clean_top}_{clean_spouse}"
                return pair_simple, pair_dated, sort_key, top_name_pure, top_dates, spouse_name, spouse_dates, lineage, len(full_line_names), id_path_str

            # Continue search down
            dates = format_date_span(curr)
            # Add *Child* to path (reversed later)
            # Actually, path_from_start accumulates from bottom up?
            # No, queue is traversing UP?
            # Wait, `analyze_lineage_deep` logic in V95 was traversing UP from a Tester?
            # No, looking at logic: `fam_id = person["parents_fam"]`. Yes, it traverses UP.

            # The name added to path needs dates
            rich_name = f"{person['name']} {dates}".strip()
            new_path = path_from_start + [rich_name]; new_ids = ids_from_start + [curr]

            fam_id = person["parents_fam"]
            if fam_id and fam_id in families:
                fam = families[fam_id]
                if fam["husb"]: queue.append((fam["husb"], new_path, new_ids))
                if fam["wife"]: queue.append((fam["wife"], new_path, new_ids))

        fail_p = individuals.get(start_id, {})
        error_msg = f"Trace Failed at: {fail_p.get('name', 'Unknown')} ({start_id})"
        return "Disconnected", "⚠️ Unlinked / Disconnected Lines", "ZZ_Disconnected", "", "", "", "", error_msg, 0, ""

    rows = []
    for uid, data in individuals.items():
        if data["mask_code"]:
            pair_simple, pair_dated, sort_key, fa1, fa1_d, fa2, fa2_d, lineage, gens, id_path = analyze_lineage_deep(uid)
            final_name = resolve_name(data["mask_code"])
            rows.append({
                "Tester-Participant-MASKED": data["mask_code"], "Tester-Participant-Unmasked": final_name, "Found Match": data["name"], "ID#": uid, "cM": data["cm"], "Spacer": "",
                "Yates DNA Ancestral Line": lineage, "Authority_FirstAncestor": pair_simple, "Authority_FirstAncestor_alpha": sort_key, "Authority_FirstAncestor_dated": pair_dated,
                "fa_1 extracted": fa1, "fa_1_Dates": fa1_d, "fa_2 extracted": fa2, "fa_2 Dates": fa2_d, "Gen_Count": gens, "Ancestral_Path_IDs": id_path
            })

    # SUFFIX-AWARE SORTER
    def get_sortable_surname_py(full_name):
        if not full_name: return "zzz"
        cleaned = re.sub(r'\b(jr\.?|sr\.?|ii|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', str(full_name), flags=re.IGNORECASE)
        cleaned = re.sub(r'[,\.]', '', cleaned)
        parts = cleaned.strip().split()
        if not parts: return "zzz"
        return parts[-1].lower()

    rows.sort(key=lambda x: get_sortable_surname_py(x["Tester-Participant-Unmasked"]))

    fieldnames = ["Tester-Participant-MASKED", "Tester-Participant-Unmasked", "Found Match", "ID#", "cM", "Spacer", "Yates DNA Ancestral Line", "Authority_FirstAncestor", "Authority_FirstAncestor_alpha", "Authority_FirstAncestor_dated", "fa_1 extracted", "fa_1_Dates", "fa_2 extracted", "fa_2 Dates", "Gen_Count", "Ancestral_Path_IDs"]

    with open(CSV_DB, "w", encoding="iso-8859-15", newline="", errors="replace") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader(); writer.writerows(rows)

    print(f"\n[SUCCESS] Engine Complete. Saved {len(rows)} verified matches to {CSV_DB}.")

print("✅ Cell 3 (Dates Added) Loaded.")

✅ Cell 3 (Dates Added) Loaded.


In [31]:
# @title [CELL 4] The Publisher (V112 - Teal Nav Restoration)
def run_publisher():
    print("="*60)
    print("      [CELL 4] PUBLISHER STARTING...")
    print("="*60)

    import os
    import sys
    import json
    import pytz
    import pandas as pd
    from datetime import datetime
    import re

    # --- 1. DEFINE NAVIGATION (Restored Teal Brand Color) ---
    # Changed background from #333 to #006064
    NAV_HTML = """
    <div class="nav-bar" style="background:#006064; padding:10px; text-align:center; font-family:'Segoe UI', sans-serif; box-shadow:0 2px 5px rgba(0,0,0,0.2);">
        <a href="ons_yates_dna_register.shtml" style="color:white; margin:0 15px; text-decoration:none; font-weight:bold; font-size:1.1em;">DNA Register</a> |
        <a href="dna_network.shtml" style="color:white; margin:0 15px; text-decoration:none; font-weight:bold; font-size:1.1em;">Network Cluster</a> |
        <a href="lineage_proof.html" style="color:#b2dfdb; margin:0 15px; text-decoration:none; font-weight:bold; font-size:1.1em;">Proof Engine</a> |
        <a href="brick_wall_buster.shtml" style="color:#b2dfdb; margin:0 15px; text-decoration:none; font-weight:bold; font-size:1.1em;">Wall Buster</a> |
        <a href="dna_dossier.html" style="color:white; margin:0 15px; text-decoration:none; font-weight:bold; font-size:1.1em;">Forensic Dossier</a> |
        <a href="data_glossary.shtml" style="color:#e0f2f1; margin:0 15px; text-decoration:none;">Glossary</a> |
        <a href="research_admin.html" style="color:#e0f2f1; margin:0 15px; text-decoration:none;">Admin Hub</a>
    </div>
    """

    # --- 2. INITIALIZE & HELPER FUNCTIONS ---
    pages_to_upload = {}

    TNG_BASE_URL = "https://yates.one-name.net/tng/verticalchart.php?personID="
    TNG_SUFFIX = "&tree=tree1&parentset=0&display=vertical&generations=15"

    def normalize_id(val):
        s = str(val).replace('@', '').strip()
        if s.isdigit(): return f"I{s}"
        return s

    def build_narrative(row):
        part_name = str(row.get('Tester-Participant-Unmasked', '')).strip()
        cm = str(row.get('cM', '0'))
        anc_dated = str(row.get('Authority_FirstAncestor_dated', 'Unknown'))
        found_match = str(row.get('Found Match', 'Unknown')).strip()
        gen_count = row.get('Gen_Count', 0)
        rid = normalize_id(row.get('ID#', ''))
        linked_found_match = f'<a href="{TNG_BASE_URL}{rid}{TNG_SUFFIX}" target="_blank"><b>{found_match}</b></a>'
        return f"{part_name} is a {cm} cM DNA match to {linked_found_match} is related via {anc_dated} back {gen_count} generations."

    def build_linked_lineage(row):
        line = str(row.get('Yates DNA Ancestral Line', ''))
        found = str(row.get('Found Match', ''))
        rid = normalize_id(row.get('ID#', ''))
        if found and rid and found in line:
            url = f"{TNG_BASE_URL}{rid}{TNG_SUFFIX}"
            link_html = f'<a href="{url}" target="_blank" style="color:#006064;text-decoration:none;font-weight:bold;">{found}</a>'
            return line.replace(found, link_html)
        return line

    def get_sortable_surname(full_name):
        if not full_name: return "zzz"
        cleaned = re.sub(r'\b(jr\.?|sr\.?|ii|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', str(full_name), flags=re.IGNORECASE)
        cleaned = re.sub(r'[,\.]', '', cleaned)
        parts = cleaned.strip().split()
        if not parts: return "zzz"
        return parts[-1].lower()

    def format_last_first(full_name):
        if not full_name: return ""
        cleaned = re.sub(r'\b(jr\.?|sr\.?|ii|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', str(full_name), flags=re.IGNORECASE)
        cleaned = re.sub(r'[,\.]', '', cleaned)
        parts = cleaned.strip().split()
        if len(parts) < 2: return full_name
        surname = parts.pop()
        firstname = " ".join(parts)
        return f"{surname.title()}, {firstname}"

    # Safe Page Maker (Uses single brackets for NAV_HTML)
    def make_page(title, body_content, db_len, active_tab, stats_bar):
        return f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>{title}</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f4f7f6;padding:20px}}.card{{background:white;padding:30px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.05);max-width:1000px;margin:20px auto;}}</style></head><body><div class="wrap"><h1 class="centerline">{title}</h1><div id="nav-slot">{stats_bar}{NAV_HTML}</div><div class="card">{body_content}</div></div></body></html>"""

    # --- 3. LOAD DATA (FIRST) ---
    CSV_DB = "engine_database.csv"
    if not os.path.exists(CSV_DB):
        print("❌ ERROR: engine_database.csv not found. Run Cell 3 first.")
        return

    df = pd.read_csv(CSV_DB, encoding="iso-8859-15")
    df['Long_Narrative'] = df.apply(build_narrative, axis=1)
    df['Linked_Tree_Line'] = df.apply(build_linked_lineage, axis=1)

    # --- 4. GENERATE STATS BAR ---
    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%B %d, %Y %-I:%M %p EST")
    stats_bar_full = f"""<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>Last updated:</strong> {timestamp} &nbsp;|&nbsp; <strong>Total Autosomal matches:</strong> {len(df):,}</div>"""
    stats_bar_html = stats_bar_full

    # --- 5. PREPARE JSON DATA ---
    ancestor_data = {}
    for alpha, group_df in df.groupby('Authority_FirstAncestor_alpha'):
        match_count = int(len(group_df))
        if match_count < 2: continue

        total_cm = int(group_df['cM'].sum())
        unique_count = int(len(group_df['Tester-Participant-Unmasked'].unique()))
        top_testers = group_df['Tester-Participant-Unmasked'].value_counts().head(3).to_dict()
        badge = "Bronze"; integrity = 25; verdict = f"This line is an **Emerging Frontier**. With {match_count} matches, the connection is real but fragile."
        if "Disconnected" in alpha or "ZZ_" in alpha: badge = "Action Required"; integrity = 0; verdict = "**Data Quality Issue.** Matches linked to disconnected profiles. Fix parent links in TNG."
        elif match_count >= 5: badge = "Silver"; integrity = 50; verdict = f"**Likely Valid.** Supported by {match_count} matches sharing {total_cm} cM."
        if match_count >= 15 and unique_count >= 3: badge = "Gold"; integrity = 80; verdict = f"**Strong Genetic Confirmation.** Anchored by {unique_count} independent tester groups."
        if match_count >= 30 and unique_count >= 10: badge = "Platinum"; integrity = 100; verdict = f"**The Platinum Standard.** Biologically confirmed beyond reasonable doubt."
        ancestor_data[alpha] = {"name": group_df.iloc[0]['Authority_FirstAncestor_dated'], "matches": match_count, "cm": total_cm, "testers": unique_count, "badge": badge, "verdict": verdict, "integrity": integrity, "list_data": top_testers}

    participant_data = {}
    for p_name, group_df in df.groupby('Tester-Participant-Unmasked'):
        match_count = int(len(group_df)); total_cm = int(group_df['cM'].sum())
        top_anc = group_df['Authority_FirstAncestor_dated'].mode()[0]
        my_ancestors = group_df['Authority_FirstAncestor_dated'].value_counts().head(3).to_dict()
        badge = "New Tester"; integrity = 25; verdict = f"**Growth Opportunity.** New entrant needing cousin recruitment."
        if match_count >= 5: badge = "Active Cousin"; integrity = 50; verdict = f"**Active Contributor.** Consistent data linking to {top_anc.split('&')[0]}."
        if match_count >= 15: badge = "Keystone Tester"; integrity = 90; verdict = f"**The Keystone Driver.** Structural pillar connecting disparate branches."
        participant_data[p_name] = {"name": p_name, "matches": match_count, "cm": total_cm, "testers": 1, "badge": badge, "verdict": verdict, "integrity": integrity, "list_data": my_ancestors}

    smart_packet_json = json.dumps({"ancestors": ancestor_data, "participants": participant_data})

    proof_db_json = df[['Authority_FirstAncestor_dated','Tester-Participant-Unmasked','cM','ID#','Yates DNA Ancestral Line', 'Ancestral_Path_IDs']].rename(columns={'Authority_FirstAncestor_dated':'ancestor','Tester-Participant-Unmasked':'participant','cM':'cm','ID#':'id','Yates DNA Ancestral Line':'lineage', 'Ancestral_Path_IDs': 'path_ids'}).to_json(orient='records')

    # --- 6. GENERATE PAGES ---

    # Shared JS
    js_tools = """
    function getSortableName(fullname) {
        if (!fullname) return "zzz";
        let clean = fullname.replace(/\\b(jr\\.?|sr\\.?|iii|iv|ii|esq\\.?|m\\.d\\.?|ph\\.d\\.?)\\b/gi, "");
        clean = clean.replace(/[\\,\\.]/g, "");
        let parts = clean.trim().split(/\\s+/);
        let surname = parts.length > 0 ? parts[parts.length - 1].toLowerCase() : "zzz";
        return surname.toLowerCase() + " " + clean.toLowerCase();
    }
    function formatName(fullname) {
        if (!fullname) return "";
        let clean = fullname.replace(/\\b(jr\\.?|sr\\.?|iii|iv|ii|esq\\.?|m\\.d\\.?|ph\\.d\\.?)\\b/gi, "");
        clean = clean.replace(/[\\,\\.]/g, "");
        let parts = clean.trim().split(/\\s+/);
        if (parts.length < 2) return fullname;
        let surname = parts.pop();
        let firstname = parts.join(" ");
        return surname.charAt(0).toUpperCase() + surname.slice(1) + ", " + firstname;
    }
    """

    # Glossary Content
    GLOSSARY_CONTENT = """
    <div style="max-width:800px; margin:0 auto; font-family:'Segoe UI', sans-serif; line-height:1.6; color:#333;">
        <h2 style="color:#006064; border-bottom:2px solid #006064; padding-bottom:10px;">Forensic Genealogy Terms</h2>
        <div style="margin-bottom:20px;">
            <h3 style="color:#e65100; margin-bottom:5px;">🤝 Forensic Handshake</h3>
            <p><strong>Definition:</strong> A confirmed genetic cross-match between a specific descendant cluster and an established, independent ancestral line.</p>
            <p style="background:#fff3e0; padding:10px; border-left:4px solid #e65100;"><strong>Why it matters:</strong> This is the "Gold Standard" of validation. It proves that a "Brick Wall" group is not isolated but is biologically integrated into a proven super-family (e.g., the George1 Yates line). If your cluster "shakes hands" with George1, they are kin.</p>
        </div>
        <div style="margin-bottom:20px;">
            <h3 style="color:#0277bd; margin-bottom:5px;">🌉 Bridge Tester</h3>
            <p><strong>Definition:</strong> A specific participant whose DNA connects two or more distinct lineages.</p>
            <p><strong>Why it matters:</strong> These individuals act as "genetic hinges." Investigating their tree often reveals the missing paper trail link between two branches. They are the priority targets for research.</p>
        </div>
        <div style="margin-bottom:20px;">
            <h3 style="color:#2e7d32; margin-bottom:5px;">🎯 Target Saturation</h3>
            <p><strong>Definition:</strong> The density of confirmed descendants for a specific ancestor.</p>
            <p><strong>Why it matters:</strong> One match is a hint; 15 matches is a statistical fact. High saturation makes the "Handshake" irrefutable. If you have 20 descendants matching the same external line, the connection is real.</p>
        </div>
        <div style="margin-bottom:20px;">
            <h3 style="color:#455a64; margin-bottom:5px;">🧱 Negative Space Analysis</h3>
            <p><strong>Definition:</strong> Using DNA to determine who you are <em>not</em> related to.</p>
            <p><strong>Why it matters:</strong> If you have 40 matches, but 0 of them connect to the "New England Yates" line, we can forensically rule that line out. This saves years of wasted research time.</p>
        </div>
    </div>
    """

    # Generate Pages
    pages_to_upload["data_glossary.shtml"] = make_page("Data Glossary", GLOSSARY_CONTENT, len(df), "glossary", stats_bar_html)
    pages_to_upload["subscribe.shtml"] = make_page("Subscribe", "<h2>Subscribe</h2><p>Subscribe content here.</p>", len(df), "subscribe", stats_bar_html)
    pages_to_upload["share_dna.shtml"] = make_page("Share Your DNA", "<h2>Share Your DNA</h2><p>Share content here.</p>", len(df), "share", stats_bar_html)
    pages_to_upload["dna_theory_of_the_case.htm"] = make_page("The Yates DNA Strategy", "<h2>Theory of the Case</h2><p>Theory content here.</p>", len(df), "theory", stats_bar_html)

    # 1. Buster
    buster_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates Brick Wall Buster</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}}.dashboard{{max-width:900px;margin:0 auto;background:white;padding:30px;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1)}}.buster-header{{background:#f57f17;color:white;padding:20px;border-radius:8px 8px 0 0;margin:-30px -30px 20px -30px;text-align:center}}.control-panel{{background:#fff3e0;padding:20px;border-radius:8px;border:1px solid #ffcc80;margin-bottom:20px}}.tabs{{display:flex;gap:10px;margin-bottom:15px;border-bottom:2px solid #ddd;}}.tab{{padding:10px 20px;cursor:pointer;background:#eee;border-radius:5px 5px 0 0;font-weight:bold;flex:1;text-align:center;}}.tab.active{{background:#f57f17;color:white;}}select{{width:100%;padding:12px;font-size:16px;border:1px solid #f57c00;border-radius:4px;margin-top:5px}}button{{background:#e65100;color:white;border:none;padding:12px 25px;font-weight:bold;border-radius:4px;cursor:pointer;margin-top:10px;font-size:16px}}button:hover{{background:#ef6c00}}.results-box{{display:none;margin-top:30px}}.finding-box{{background:#e8eaf6;border-left:5px solid #3f51b5;padding:15px;margin-bottom:15px}}.conclusion-box{{background:#e0f2f1;border-left:5px solid #009688;padding:15px;margin-bottom:15px}}.speculation-box{{background:#ffebee;border-left:5px solid #e91e63;padding:15px;margin-bottom:15px}}.checkbox-list{{height:250px;overflow-y:scroll;border:1px solid #f57c00;background:white;border-radius:4px;padding:10px;}}.checkbox-item{{display:block;margin-bottom:5px;padding:5px;border-bottom:1px solid #eee;}}.checkbox-item:hover{{background-color:#ffe0b2;}}.checkbox-item input{{margin-right:10px;transform:scale(1.2);}}.tooltip{{cursor:pointer;color:#0277bd;font-weight:bold;margin-left:10px;font-size:1.2em;}}.info-bubble{{display:none;background:#e1f5fe;border:1px solid #81d4fa;padding:15px;border-radius:8px;margin-top:10px;color:#01579b;font-size:0.95em;line-height:1.5;}}</style></head><body><div class="wrap"><div class="dashboard"><div class="buster-header"><h1>🧱 Brick Wall Buster (v112)</h1><p>Predictive Forensic Analysis for Stalled Lineages</p></div>{stats_bar_full}{NAV_HTML}{SITE_INFO}
    <div style="background:#e3f2fd;padding:15px;border:1px solid #90caf9;border-radius:5px;margin-bottom:20px;display:flex;align-items:center;justify-content:space-between;">
        <div><strong>🔎 Smart ID Search:</strong> Type a TNG ID (e.g. <em>I55</em>) to analyze descendant cluster.</div>
        <div><input type="text" id="smartIdInput" placeholder="I..." style="padding:5px;width:100px;"> <button onclick="runSmartSearch()" style="padding:5px 10px;font-size:14px;background:#0277bd;margin-top:0;">Go</button></div>
    </div>
    <div class="tabs"><div class="tab active" onclick="setMode('ancestor')">1. Define by Ancestor</div><div class="tab" onclick="setMode('custom')">2. Define by Group</div><div class="tab" onclick="setMode('focus')">3. Analyze Specific Participant</div></div><div class="control-panel"><div id="panel-ancestor"><label style="font-weight:bold;color:#bf360c;">Option A: Select Your "End of Line" Ancestor: <span class="tooltip" onclick="toggleInfo('a')">❓</span></label><div id="info-a" class="info-bubble"><strong>Top-Down Analysis:</strong><br>Select a known ancestor to see every DNA-confirmed descendant in the study. This validates specific branches and shows you exactly who carries the DNA of that lineage.</div><select id="wallSelect"><option value="">-- Choose the ancestor you are stuck on --</option></select></div><div id="panel-custom" style="display:none;"><label style="font-weight:bold;color:#bf360c;">Option B: Select Multiple Testers (Scroll & Check): <span class="tooltip" onclick="toggleInfo('b')">❓</span></label><div id="info-b" class="info-bubble"><strong>The Negative Space Strategy:</strong><br>1. <strong>Isolate:</strong> We identify your specific group (the "Cluster").<br>2. <strong>Compare:</strong> We compare your cluster's size against the rest of the database.<br>3. <strong>Predict:</strong> We calculate the statistical probability of your cluster belonging to one of the major Yates lines based on their dominance in the study.</div><div id="testerList" class="checkbox-list"></div></div><div id="panel-focus" style="display:none;"><label style="font-weight:bold;color:#bf360c;">Option C: Select a Single Participant to Predict Placement: <span class="tooltip" onclick="toggleInfo('c')">❓</span></label><div id="info-c" class="info-bubble"><strong>Point-to-Point Analysis (The Handshake):</strong><br>Select one specific tester. The tool scans their matches to find the strongest "Handshake" with a proven line. We look for <strong>Target Saturation</strong> (do they match 40+ descendants?) to predict where they fit.</div><select id="focusSelect"><option value="">-- Choose a Participant --</option></select></div><div style="margin-top:10px;display:flex;justify-content:space-between;"><button onclick="runAnalysis()">🔨 Bust This Wall</button><button onclick="resetAll()" style="background:#d32f2f;">✕ Clear Selections</button></div></div><div id="results" class="results-box"><div class="no-print" style="float:right;"><button onclick="window.print()" style="background:#455a64;font-size:14px;color:white;border:none;padding:8px 15px;border-radius:4px;cursor:pointer;">🖨️ Print Report</button></div><h2 style="border-bottom:2px solid #ddd;padding-bottom:10px;margin-top:0;">Forensic Report</h2><div class="finding-box"><strong>🔍 Findings:</strong> <span id="txt-finding"></span></div><div class="conclusion-box"><strong>💡 Study Context:</strong> <span id="txt-conclusion"></span></div><div class="speculation-box"><strong>🤝 Forensic Handshake Analysis:</strong> <span id="txt-speculation"></span></div><div id="bridge-alert" style="display:none;background:#c8e6c9;border:1px solid #4caf50;padding:15px;margin-top:15px;color:#2e7d32;"></div></div></div></div><script>const DATA={smart_packet_json};const FULL_DB={proof_db_json};let MODE='ancestor';
    let SMART_NAME = "";
    let SMART_ID = "";
    {js_tools}
    function toggleInfo(id){{
        const b=document.getElementById('info-'+id);
        const current = b.style.display;
        document.querySelectorAll('.info-bubble').forEach(x=>x.style.display='none');
        b.style.display=(current==='block')?'none':'block';
    }}
    const ancSel=document.getElementById('wallSelect');Object.keys(DATA.ancestors).sort((a,b)=>DATA.ancestors[a].name.localeCompare(DATA.ancestors[b].name)).forEach(k=>{{const o=document.createElement('option');o.value=k;o.innerText=DATA.ancestors[k].name;ancSel.appendChild(o);}});
    const testListDiv=document.getElementById('testerList');
    const focusSel=document.getElementById('focusSelect');
    const allTesters=[...new Set(FULL_DB.map(r=>r.participant))];
    allTesters.sort((a,b)=> getSortableName(a).localeCompare(getSortableName(b)));

    let listHTML="";
    allTesters.forEach(t=>{{
        const fmt = formatName(t);
        listHTML+=`<label class="checkbox-item"><input type="checkbox" value="${{t}}"> ${{fmt}}</label>`;
        const opt = document.createElement('option'); opt.value = t; opt.innerText = fmt; focusSel.appendChild(opt);
    }});
    testListDiv.innerHTML=listHTML;

    function setMode(m){{MODE=m;document.querySelectorAll('.tab').forEach(t=>t.classList.remove('active'));event.target.classList.add('active');
        document.getElementById('panel-ancestor').style.display='none';
        document.getElementById('panel-custom').style.display='none';
        document.getElementById('panel-focus').style.display='none';
        document.getElementById('panel-'+m).style.display='block';
        document.getElementById('results').style.display='none';
    }}

    function resetAll(){{
        document.getElementById('results').style.display='none';
        ancSel.selectedIndex=0;
        focusSel.selectedIndex=0;
        document.querySelectorAll('#testerList input:checked').forEach(cb=>cb.checked=false);
        SMART_NAME = "";
        SMART_ID = "";
        document.title = "Yates Brick Wall Buster";
    }}

    function runSmartSearch(){{
        let idVal = document.getElementById('smartIdInput').value.trim().toUpperCase();
        if(!idVal) return;
        // Correctly escaped for Python: \\d
        if (/^\\d+$/.test(idVal)) idVal = 'I' + idVal;
        SMART_ID = idVal;

        const descendants = FULL_DB.filter(r => (r.id === idVal) || (r.path_ids && r.path_ids.toUpperCase().includes(idVal)));

        if(descendants.length === 0) {{
            alert(`ID ${{idVal}} not found in database.`);
            return;
        }}

        const refRow = descendants.find(r => r.path_ids && r.path_ids.toUpperCase().includes(idVal));
        if (refRow) {{
            const ids = refRow.path_ids.split(",");
            const names = refRow.lineage.split(" -> ");
            const idx = ids.findIndex(x => x.toUpperCase() === idVal);
            if (idx !== -1 && names[idx]) {{
                // This name should now include dates from Cell 3
                SMART_NAME = names[idx];
            }} else {{
                SMART_NAME = idVal;
            }}
        }} else {{
            SMART_NAME = idVal;
        }}

        setMode('custom');
        document.querySelectorAll('#testerList input:checked').forEach(cb=>cb.checked=false);

        const targets = descendants.map(d => d.participant);
        document.querySelectorAll('#testerList input').forEach(cb => {{
            if(targets.includes(cb.value)) cb.checked = true;
        }});

        runAnalysis();
    }}

    function runAnalysis(){{
        let targetName="",clusterCount=0,clusterMembers=[],clusterMatches=0;
        let fileTitle = "Yates_Report";

        if(MODE==='custom'){{
            const checkedBoxes=document.querySelectorAll('#testerList input:checked');
            if(checkedBoxes.length===0)return alert("Select at least one tester.");
            clusterMembers=Array.from(checkedBoxes).map(cb=>cb.value);

            // V107 Logic: Smart Search Display
            if (SMART_NAME && SMART_ID) {{
                targetName = `<strong>${{SMART_NAME}}</strong> <br><span style='font-size:0.9em;color:#666;'>(${{clusterMembers.length}} Descendants)</span>`;
                let safeName = SMART_NAME.replace(/[^a-z0-9]/gi, '_').replace(/_+/g, '_');
                fileTitle = `${{SMART_ID}}_${{safeName}}_wallbust`;
            }} else {{
                targetName = "Custom Group ("+clusterMembers.length+" Testers)";
                fileTitle = "Custom_Group_wallbust";
            }}

            clusterCount=clusterMembers.length;
            const groupRows=FULL_DB.filter(r=>clusterMembers.includes(r.participant));
            clusterMatches=groupRows.length;
        }} else if(MODE==='focus') {{
            const fVal = document.getElementById('focusSelect').value;
            if(!fVal) return alert("Select a participant.");
            clusterMembers=[fVal];
            targetName="Focus: " + formatName(fVal);
            clusterCount=1;
            const groupRows=FULL_DB.filter(r=>r.participant === fVal);
            clusterMatches=groupRows.length;
            fileTitle = formatName(fVal).replace(/[^a-z0-9]/gi, '_') + "_wallbust";
        }} else {{
            const key=ancSel.value;if(!key)return;
            const d=DATA.ancestors[key];
            targetName=d.name;clusterCount=d.testers;
            clusterMembers=Object.keys(d.list_data);
            clusterMatches=d.matches;
            fileTitle = d.name.split(' ')[0] + "_Yates_wallbust";
        }}

        document.title = fileTitle;

        const allMatches=FULL_DB.length,outsideMatches=allMatches-clusterMatches,pct=((clusterMatches/allMatches)*100).toFixed(2);

        let bridgeHTML="";
        const clusterRows=FULL_DB.filter(r=>clusterMembers.includes(r.participant));
        const bridges=clusterRows.filter(r=>r.ancestor!==targetName&&r.ancestor!==null);
        if(bridges.length>0){{
            const uniqueBridges=[...new Set(bridges.map(b=>formatName(b.participant)+" -> "+b.ancestor))];

            if(uniqueBridges.length>0){{
                const totalBridges = uniqueBridges.length;
                bridgeHTML=`<div style="margin-bottom:5px;"><strong>🌉 BRIDGE DETECTED:</strong> We found <strong>${{totalBridges}}</strong> instances where this group connects to other lines. Here are the top 5 examples:</div><ul>`;
                uniqueBridges.slice(0,5).forEach(b=>bridgeHTML+=`<li>${{b}}</li>`);
                if(totalBridges > 5) bridgeHTML+=`<li><em>...and ${{totalBridges - 5}} more connections.</em></li>`;
                bridgeHTML+=`</ul>`;
                document.getElementById('bridge-alert').innerHTML=bridgeHTML;
                document.getElementById('bridge-alert').style.display='block';
            }}else{{document.getElementById('bridge-alert').style.display='none';}}
        }}else{{document.getElementById('bridge-alert').style.display='none';}}

        const memStr=(clusterMembers.length>5)?clusterMembers.slice(0,5).join(", ")+"...":clusterMembers.join(", ");
        let findHTML=`<ul style="margin-top:0;padding-left:20px;"><li><strong>Target:</strong> ${{targetName}}</li><li><strong>Participants:</strong> ${{clusterCount}}</li><li><strong>Data Volume:</strong> ${{clusterMatches}} confirmed matches.</li></ul>`;
        document.getElementById('txt-finding').innerHTML=findHTML;
        document.getElementById('txt-conclusion').innerHTML=`This cluster represents <strong>${{pct}}%</strong> of the total Yates database. There are <strong>${{outsideMatches.toLocaleString()}} other matches</strong> in the study.`;

        let otherGroups=[];
        if (MODE !== 'ancestor') {{
             let counts = {{}};
             clusterRows.forEach(r => {{ if (r.ancestor) {{ counts[r.ancestor] = (counts[r.ancestor] || 0) + 1; }} }});
             otherGroups = Object.entries(counts).map(([k,v]) => ({{name: k, matches: v}}));
        }} else {{
             for(const[k,v]of Object.entries(DATA.ancestors)){{if(v.name!==targetName&&v.matches>=5)otherGroups.push(v);}}
        }}

        otherGroups.sort((a,b)=>b.matches-a.matches);
        const top3=otherGroups.slice(0,3);

        let html="";
        if(top3.length===0){{
            html+="<p>No strong signals found. The database currently lacks a 'Proven Line' (5+ matches) that connects to your selection.</p>";
        }}else{{
            html+=`<p>We analyzed the database dominance to find the strongest 'Handshakes' (Cross-Matches) where your cluster connects with a proven line:</p><ul>`;
            top3.forEach(g=>{{
                html+=`<li>We found <strong>${{g.matches}} instances</strong> where a DNA kit in your Brick Wall Cluster shared significant centimorgans (cM) with a DNA kit in the <strong>${{g.name}}</strong> lineage. (Probability: <span style="color:green">High</span>)</li>`;
            }});
            html+=`</ul>`;
        }}
        document.getElementById('txt-speculation').innerHTML=html;
        document.getElementById('results').style.display='block';
    }}</script></body></html>"""
    pages_to_upload["brick_wall_buster.shtml"] = buster_html

    # 2. Lineage Proof Engine
    proof_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Lineage Proof Engine</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}}.proof-card{{background:white;max-width:1000px;margin:20px auto;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);padding:30px}}.badge-large{{font-size:1.2em;padding:10px 20px;border-radius:30px;font-weight:bold;color:white;display:inline-block;margin-bottom:20px}}.badge-platinum{{background:#e5e4e2;color:#333;border:2px solid #ccc}}.badge-gold{{background:#ffd700;color:#b45f06}}.badge-silver{{background:#c0c0c0;color:#333}}.badge-bronze{{background:#cd7f32}}.stats-grid{{display:grid;grid-template-columns:repeat(auto-fit,minmax(150px,1fr));gap:15px;margin-bottom:30px}}.stat-box{{background:#f8f9fa;padding:15px;border-radius:8px;text-align:center;border:1px solid #ddd}}.stat-val{{font-size:1.8em;font-weight:bold;color:#006064}}.stat-lbl{{color:#666;font-size:0.9em;text-transform:uppercase}}table{{width:100%;border-collapse:collapse;margin-top:20px}}th{{background:#004d40;color:white;padding:12px;text-align:left}}td{{padding:10px;border-bottom:1px solid #eee}}tr:hover{{background-color:#f1f8e9}}select{{padding:10px;font-size:16px;width:100%;max-width:600px;margin-bottom:20px;border:1px solid #006064;border-radius:4px}}</style></head><body><div class="wrap"><h1 class="centerline">🧬 Lineage Proof Engine</h1><div id="nav-slot">{stats_bar_html}{NAV_HTML}</div><div class="proof-card">
    <div style="background:#e3f2fd;padding:20px;border-radius:8px;margin-bottom:30px;border:1px solid #90caf9;">
        <h3 style="color:#0277bd;margin-top:0;">🔎 Deep Search by ID#</h3>
        <p>Type a TNG ID (e.g., <strong>I55</strong>) to find all cousins who descend from that specific person.</p>
        <input type="text" id="idSearchInput" placeholder="Enter ID (e.g. I1234)..." style="padding:10px;width:200px;border:1px solid #0277bd;border-radius:4px;">
        <button onclick="runIdSearch()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;cursor:pointer;font-weight:bold;">Find Descendants</button>
        <div id="id-results" style="margin-top:15px;display:none;"></div>
    </div>

    <div class="no-print" style="float:right;"><button onclick="window.print()" style="background:#455a64;font-size:14px;padding:8px 15px;color:white;border:none;border-radius:4px;cursor:pointer;">🖨️ Print Report</button></div>
    <h3 style="color:#006064;margin-top:0;">Verify an Ancestral Line</h3><p>Select a couple to audit the forensic evidence supporting them.</p><select id="proofSelect" onchange="runProof()"><option value="">-- Select Ancestor --</option></select><div id="proof-result" style="display:none;"><div style="text-align:center"><span id="p-badge" class="badge-large"></span></div><div class="stats-grid"><div class="stat-box"><div class="stat-val" id="p-matches">0</div><div class="stat-lbl">Matches</div></div><div class="stat-box"><div class="stat-val" id="p-testers">0</div><div class="stat-lbl">Unique Testers</div></div><div class="stat-box"><div class="stat-val" id="p-cm">0</div><div class="stat-lbl">Total cM</div></div><div class="stat-box"><div class="stat-val" id="p-integrity">0%</div><div class="stat-lbl">Integrity</div></div></div><div style="background:#e0f2f1;padding:15px;border-left:5px solid #004d40;margin-bottom:20px;font-style:italic;" id="p-verdict"></div><h4 style="border-bottom:2px solid #ddd;padding-bottom:10px;">Evidence Manifest</h4><div style="max-height:500px;overflow-y:auto;"><table id="evidence-table"><thead><tr><th>Participant</th><th>cM</th><th>Lineage Path</th></tr></thead><tbody></tbody></table></div></div></div></div><script>const DATA={smart_packet_json};const DB={proof_db_json};
    const sel=document.getElementById('proofSelect');
    Object.keys(DATA.ancestors).sort((a,b)=>DATA.ancestors[a].name.localeCompare(DATA.ancestors[b].name)).forEach(k=>{{const o=document.createElement('option');o.value=k;o.innerText=DATA.ancestors[k].name;sel.appendChild(o);}});

    function runProof(){{
        const key=sel.value;if(!key){{document.getElementById('proof-result').style.display='none';return;}}const d=DATA.ancestors[key];document.getElementById('proof-result').style.display='block';
        document.getElementById('p-badge').className='badge-large badge-'+d.badge.toLowerCase().split(' ')[0];document.getElementById('p-badge').innerText=d.badge;document.getElementById('p-matches').innerText=d.matches;document.getElementById('p-testers').innerText=d.testers;document.getElementById('p-cm').innerText=d.cm.toLocaleString();document.getElementById('p-integrity').innerText=d.integrity+'%';document.getElementById('p-verdict').innerHTML=d.verdict;
        const matches=DB.filter(r=>r.ancestor===d.name).sort((a,b)=>b.cm-a.cm);const tbody=document.querySelector('#evidence-table tbody');tbody.innerHTML='';matches.forEach(m=>{{const tr=document.createElement('tr');tr.innerHTML=`<td>${{m.participant}}</td><td>${{m.cm}}</td><td style="font-size:0.9em;color:#555;">${{m.lineage}}</td>`;tbody.appendChild(tr);}});
    }}

    function runIdSearch(){{
        let idVal = document.getElementById('idSearchInput').value.trim().toUpperCase();
        if(!idVal) return;
        if (/^\\d+$/.test(idVal)) idVal = 'I' + idVal;

        const resDiv = document.getElementById('id-results');
        const hits = DB.filter(r => r.path_ids && r.path_ids.toUpperCase().includes(idVal));
        if(hits.length === 0) {{
            resDiv.innerHTML = `<span style="color:red;font-weight:bold;">No descendants found for ID: ${{idVal}} (checked ${{DB.length}} records)</span>`;
        }} else {{
            let html = `<strong>Found ${{hits.length}} descendants for ${{idVal}}:</strong><ul style="margin-top:5px;">`;
            hits.forEach(h => html += `<li><strong>${{h.participant}}</strong> (${{h.cm}} cM) - via ${{h.ancestor}}</li>`);
            html += `</ul>`;
            resDiv.innerHTML = html;
        }}
        resDiv.style.display = 'block';
    }}
    </script></body></html>"""
    pages_to_upload["lineage_proof.html"] = proof_html

    # 3. Dossier
    dossier_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates DNA Forensic Dossier</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}}.dossier-card{{background:white;max-width:900px;margin:20px auto;border-radius:8px;box-shadow:0 4px 20px rgba(0,0,0,0.1);overflow:hidden;position:relative}}.dossier-header{{background:#006064;color:white;padding:25px;text-align:center}}.dossier-body{{padding:30px}}.badge{{display:inline-block;padding:8px 15px;border-radius:20px;font-weight:bold;color:white;font-size:0.9em;margin-bottom:15px;text-transform:uppercase}}.badge-platinum, .badge-keystone{{background:#e5e4e2;color:#333;border:2px solid #ccc}}.badge-gold{{background:#ffd700;color:#b45f06}}.badge-silver, .badge-active{{background:#c0c0c0;color:#333}}.badge-bronze, .badge-new{{background:#cd7f32}}.metric-grid{{display:grid;grid-template-columns:1fr 1fr;gap:20px;margin-bottom:30px}}.metric-box{{background:#f8f9fa;padding:20px;border-radius:8px;text-align:center;border:1px solid #e9ecef}}.metric-val{{font-size:2em;font-weight:bold;color:#004d40}}.metric-lbl{{color:#666;font-size:0.9em;text-transform:uppercase}}.progress-container{{margin:20px 0}}.progress-bar{{background:#e9ecef;height:10px;border-radius:5px;overflow:hidden}}.progress-fill{{height:100%;background:#00838f;width:0%;transition:width 1s ease}}.verdict-box{{background:#e0f2f1;border-left:5px solid #004d40;padding:20px;margin:20px 0;font-family:'Georgia',serif;font-style:italic;color:#004d40;line-height:1.5}}.contributors{{margin-top:20px;border-top:1px solid #eee;padding-top:20px}}.contributor-item{{display:flex;justify-content:space-between;padding:8px 0;border-bottom:1px solid #f1f1f1}}select{{padding:10px;font-size:16px;width:100%;max-width:500px;margin:20px auto;display:block}}.switch{{text-align:center;margin:20px}}.switch label{{margin:0 15px;font-weight:bold;cursor:pointer;color:#006064}}.action-btn{{padding:10px 20px;background:#00838f;color:white;border:none;border-radius:4px;cursor:pointer;font-size:16px;margin:0 5px}}.action-btn:hover{{background:#006064}}#composite-container{{background:white;max-width:900px;margin:20px auto;padding:20px;border-radius:8px;box-shadow:0 4px 20px rgba(0,0,0,0.1);display:none;}}.comp-table{{width:100%;border-collapse:collapse;}}.comp-table th{{background:#004d40;color:white;padding:12px;text-align:left;}}.comp-table td{{padding:10px;border-bottom:1px solid #eee;}}.footer-total{{margin-top:15px;text-align:right;font-weight:bold;color:#004d40;border-top:2px solid #eee;padding-top:10px;}}</style></head><body><div class="wrap"><h1 class="centerline">Yates DNA Forensic Dossier (v80)</h1><div id="nav-slot">{stats_bar_html}{NAV_HTML}</div><div class="switch"><label><input type="radio" name="mode" value="ancestor" checked onchange="switchMode()"> Search by Ancestor</label><label><input type="radio" name="mode" value="participant" onchange="switchMode()"> Search by Participant</label></div><div style="text-align:center;margin:30px;"><select id="dossierSelect"><option value="">-- Select --</option></select><div style="margin-top:15px;"><button class="action-btn" onclick="addReport()">Add Report</button><button class="action-btn" style="background:#d32f2f;" onclick="clearReports()">Clear All</button></div></div><div id="composite-container"><div class="no-print" style="float:right;"><button onclick="window.print()" style="background:#455a64;font-size:14px;padding:8px 15px;color:white;border:none;border-radius:4px;cursor:pointer;">🖨️ Print Dossier</button></div><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">Comparison Dashboard</h2><table class="comp-table"><thead><tr><th>Name</th><th>Role/Badge</th><th>Matches</th><th>Total cM</th><th>Integrity</th></tr></thead><tbody id="comp-body"></tbody></table><div id="comp-footer" class="footer-total"></div></div><div id="report-stack"></div></div><script>const DATA={smart_packet_json};let currentMode = 'ancestor';
    let compTotalMatches = 0; let compTotalCM = 0; let compCount = 0;
    {js_tools}
    function switchMode(){{currentMode = document.querySelector('input[name="mode"]:checked').value;populateDropdown();clearReports();}}
    function populateDropdown(){{
        const sel = document.getElementById('dossierSelect');
        sel.innerHTML = '<option value="">-- Select --</option>';
        const source = (currentMode === 'ancestor') ? DATA.ancestors : DATA.participants;
        const sortedKeys = Object.keys(source).sort((a,b) => {{
            const nA = source[a].name; const nB = source[b].name;
            if (currentMode === 'participant') {{ return getSortableName(nA).localeCompare(getSortableName(nB)); }}
            return nA.localeCompare(nB);
        }});
        sortedKeys.forEach(key => {{ const opt = document.createElement('option'); opt.value = key; opt.innerText = (currentMode === 'participant') ? formatName(source[key].name) : source[key].name; sel.appendChild(opt); }});
    }}
    function clearReports(){{document.getElementById('report-stack').innerHTML = '';document.getElementById('comp-body').innerHTML = '';document.getElementById('composite-container').style.display = 'none';document.getElementById('dossierSelect').selectedIndex = 0; compTotalMatches=0; compTotalCM=0; compCount=0;}}
    function addReport(){{const key = document.getElementById('dossierSelect').value;if(!key) return;const d = (currentMode === 'ancestor') ? DATA.ancestors[key] : DATA.participants[key];document.getElementById('composite-container').style.display = 'block';
    compTotalMatches += d.matches; compTotalCM += d.cm; compCount++;
    const tr = document.createElement('tr');tr.innerHTML = `<td><b>${{d.name}}</b></td><td>${{d.badge}}</td><td>${{d.matches}}</td><td>${{d.cm.toLocaleString()}}</td><td>${{d.integrity}}%</td>`;document.getElementById('comp-body').appendChild(tr);
    document.getElementById('comp-footer').innerText = `Composite Evidence: ${{compCount}} Profiles, ${{compTotalMatches.toLocaleString()}} Matches, ${{compTotalCM.toLocaleString()}} cM`;
    const badgeClass = d.badge.toLowerCase().split(' ')[0];const lblTesters = (currentMode==='ancestor') ? 'Unique Testers' : 'Study Rank';const listTitle = (currentMode === 'ancestor') ? 'Top Contributors' : 'Ancestral Connections';let listHTML = '';for (const [name, count] of Object.entries(d.list_data)) {{listHTML += `<div class="contributor-item"><span>${{name}}</span><span>${{count}} matches</span></div>`;}}const html = `<div class="dossier-card" style="display:block; animation: fadeIn 0.5s;"><div class="dossier-header"><h2 style="margin:0">${{d.name}}</h2><div style="margin-top:10px;opacity:0.9">${{currentMode.toUpperCase()}} REPORT</div></div><div class="dossier-body"><div style="text-align:center"><span class="badge badge-${{badgeClass}}">${{d.badge}}</span></div><div class="verdict-box"><strong>Forensic Analysis:</strong><br>${{d.verdict}}</div><div class="metric-grid"><div class="metric-box"><div class="metric-val">${{d.matches}}</div><div class="metric-lbl">Matches</div></div><div class="metric-box"><div class="metric-val">${{d.testers}}</div><div class="metric-lbl">${{lblTesters}}</div></div><div class="metric-box"><div class="metric-val">${{d.cm.toLocaleString()}}</div><div class="metric-lbl">Total cM</div></div><div class="metric-box"><div class="metric-val">${{d.integrity}}%</div><div class="metric-lbl">Integrity Score</div></div></div><div class="progress-container"><div style="display:flex;justify-content:space-between;margin-bottom:5px;font-size:0.9em;font-weight:bold;"><span>Score</span><span>${{d.integrity}}/100</span></div><div class="progress-bar"><div class="progress-fill" style="width:${{d.integrity}}%"></div></div></div><div class="contributors"><h3>${{listTitle}}</h3><div>${{listHTML}}</div></div></div></div>`;document.getElementById('report-stack').insertAdjacentHTML('afterbegin', html);}}populateDropdown();</script></body></html>"""
    pages_to_upload["dna_dossier.html"] = dossier_html

    # 4. Guide (Uses stats_bar_html)
    contents_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates Study User Guide</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f4f7f6;padding:20px}}.guide-grid{{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:25px;max-width:1200px;margin:30px auto}}.guide-card{{background:white;padding:25px;border-radius:8px;border-left:5px solid #006064;box-shadow:0 4px 10px rgba(0,0,0,0.05);transition:transform 0.2s}}.guide-card:hover{{transform:translateY(-5px)}}.card-title{{font-size:1.4em;font-weight:bold;color:#004d40;margin-top:0}}.card-why{{color:#b71c1c;font-weight:bold;margin:10px 0 5px 0;font-size:0.9em;text-transform:uppercase}}.card-what{{color:#555;font-size:1em;line-height:1.5;margin-bottom:20px}}.card-btn{{display:inline-block;padding:10px 20px;background:#00838f;color:white;text-decoration:none;border-radius:4px;font-weight:bold}}.card-btn:hover{{background:#006064}}</style></head><body><div class="wrap"><h1 class="centerline">Welcome to the Yates DNA Study Portal</h1><div id="nav-slot">{stats_bar_html}{NAV_HTML}</div><div style="text-align:center;max-width:800px;margin:20px auto;color:#444;font-size:1.1em;">This site transforms raw DNA data into forensic genealogical evidence. Use the tools below to explore your heritage, verify ancestors, and analyze the strength of your genetic connections.</div>
    <div class="guide-grid">
        <div class="guide-card"><h2 class="card-title">1. The DNA Register</h2><div class="card-why">Why View This?</div><div class="card-what">To see the raw evidence. This is the master list of all 1,700+ DNA matches in the study, sorted by ancestral line.</div><a href="ons_yates_dna_register.shtml" class="card-btn">Open Register</a></div>
        <div class="guide-card"><h2 class="card-title">2. DNA Network</h2><div class="card-why">Why View This?</div><div class="card-what">To see the big picture. Visual clusters showing which ancestral lines are genetically "saturated" and proven by multiple testers.</div><a href="dna_network.shtml" class="card-btn">View Network</a></div>
        <div class="guide-card"><h2 class="card-title">3. Lineage Proof Engine</h2><div class="card-why">Why View This?</div><div class="card-what">To verify a connection. An interactive tool that tests if a specific ancestor is biologically confirmed by independent cousins.</div><a href="lineage_proof.html" class="card-btn">Run Proof</a></div>
        <div class="guide-card" style="border-left-color:#f57f17;"><h2 class="card-title" style="color:#e65100;">4. Brick Wall Buster</h2><div class="card-why" style="color:#bf360c;">Why View This?</div><div class="card-what">To break through a dead end. This predictive engine uses "Negative Space Analysis" to suggest which proven family line you likely belong to based on who you match.</div><a href="brick_wall_buster.shtml" class="card-btn" style="background:#ef6c00;">Bust This Wall</a></div>
        <div class="guide-card"><h2 class="card-title">5. Forensic Dossier</h2><div class="card-why">Why View This?</div><div class="card-what">To get your "Scorecard." Generate a one-page forensic report on yourself or an ancestor, grading the strength of the evidence.</div><a href="dna_dossier.html" class="card-btn">Create Dossier</a></div>
        <div class="guide-card"><h2 class="card-title">6. Research Admin Hub</h2><div class="card-why">Why View This?</div><div class="card-what">For study managers. A high-level audit showing participant statistics, masked IDs, and total study metrics.</div><a href="research_admin.html" class="card-btn" style="background:#455a64;">Admin Access</a></div>
        <div class="guide-card"><h2 class="card-title">7. Data Glossary</h2><div class="card-why">Why View This?</div><div class="card-what">To understand the terms. Definitions for "Keystone Tester," "Platinum Standard," and other forensic terms used in this study.</div><a href="data_glossary.shtml" class="card-btn" style="background:#78909c;">Read Glossary</a></div>
        <div class="guide-card" style="border-left-color:#fbc02d;"><h2 class="card-title">8. Corrections & Mysteries</h2><div class="card-why">See Something? Say Something.</div><div class="card-what">We know we aren't 100% right, nor 100% wrong. Genealogy is a collaboration. If you can solve a mystery or correct a detail, tell us. <br><br><strong>Important:</strong> Please copy/paste the <strong>ID# (e.g., I1234)</strong> so we can fix the correct person.</div><a href="mailto:yates@one-name.org?subject=Yates DNA Study: Correction / Mystery&body=I have a correction or potential update regarding Person ID: [PASTE ID HERE]. Here are the details:" class="card-btn" style="background:#f9a825;color:#333;">Email Correction</a></div>
    </div></div></body></html>"""
    pages_to_upload["contents.shtml"] = contents_html

    # --- 7. UPLOAD ---
    print("\n[STEP 3] Uploading to Server...")
    if len(pages_to_upload) > 0:
        try:
            ftps = connect_session()
            target_dirs = [f"/{REMOTE_SUBDIR}", f"/public_html/{REMOTE_SUBDIR}", f"/public_html/yates.one-name.net/{REMOTE_SUBDIR}", f"htdocs/{REMOTE_SUBDIR}", REMOTE_SUBDIR]
            found_dir = False
            for d in target_dirs:
                try:
                    ftps.cwd(d)
                    found_dir = True
                    print(f"[SUCCESS] Locked onto correct web directory: {d}")
                    break
                except: pass

            if not found_dir:
                print(f"\n[CRITICAL WARNING] Could not locate the '{REMOTE_SUBDIR}' folder!")
                print(f"STUCK IN: {ftps.pwd()}")
                print("Aborting upload to avoid ghost files.")
            else:
                for fn, content in pages_to_upload.items():
                    with open(fn, "w", encoding="utf-8") as f: f.write(content)
                    with open(fn, "rb") as fh: ftps.storbinary(f"STOR {fn}", fh)
                    print(f"    - Uploaded: {fn}")
                print(f"\n[SUCCESS] All Files Published Live to {ftps.pwd()}")
            ftps.quit()
        except Exception as e: print(f"\n[ERROR] Upload Failed: {e}")
    else: print("[WARN] No content generated to upload.")

print("✅ Cell 4 (Publisher) Loaded.")

✅ Cell 4 (Publisher) Loaded.


In [32]:
# @title [CELL 5] MASTER ORCHESTRATOR (Run This Button)
import os, sys
print("="*60)
print("      MASTER ORCHESTRATOR (V81)")
print("      (Running Engine -> Publisher -> Upload)")
print("="*60)

if 'run_engine' not in globals() or 'run_publisher' not in globals():
    print("❌ ERROR: Modules not loaded! Please run Cells 3 and 4 first.")
else:
    print("\n>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...")
    try:
        run_engine()
        print("✅ PHASE 1 COMPLETE.")

        print("\n>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...")
        run_publisher()
        print("✅ PHASE 2 COMPLETE.")

        print("\n" + "="*60)
        print("      🏆 V81 UPDATE SUCCESSFUL")
        print("="*60)
    except Exception as e:
        print(f"\n❌ CRITICAL FAILURE: {e}")

      MASTER ORCHESTRATOR (V81)
      (Running Engine -> Publisher -> Upload)

>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...
      [CELL 3] ENGINE STARTING (V102)
      (Features: Dates embedded in Lineage Paths)

[STEP 0] Checking for Required Files...
    ✅ Local files found. Priority 1.
[INFO] Using GEDCOM: yates_study_2025.ged

[STEP 2] Unmasking Participants...
    - Loaded 94 privacy keys.
    - Generated yates_study_2025_UNMASKED.ged

[STEP 3] Tracing Lineages...
    - Parsed 63667 individuals.

[SUCCESS] Engine Complete. Saved 1710 verified matches to engine_database.csv.
✅ PHASE 1 COMPLETE.

>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...
      [CELL 4] PUBLISHER STARTING...

[STEP 3] Uploading to Server...
[SUCCESS] Locked onto correct web directory: /ons-study
    - Uploaded: data_glossary.shtml
    - Uploaded: subscribe.shtml
    - Uploaded: share_dna.shtml
    - Uploaded: dna_theory_of_the_case.htm
    - Uploaded: brick_wall_buster.shtml
    - Uploaded: lineage_proof.html
    - U